<a href="https://colab.research.google.com/github/FaQ2108/Trading-Algoritmico-con-SmallCaps/blob/main/Limpieza_y_Filtrado_bbdd_(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install sqlalchemy pymysql h5py google-colab

from getpass import getpass  # Importar getpass para ocultar la entrada de la contraseña
import pandas as pd
import sqlalchemy as db
from sqlalchemy.exc import SQLAlchemyError
import h5py
from google.colab import drive
import os

class DatabaseAnalyzer:
    def __init__(self):
        self.engine = None
        self.cleaned_dataframes = {}

    def get_db_connection(self):
        try:
            host = input("Introduce el host de la base de datos: ")
            user = input("Introduce el usuario de la base de datos: ")
            password = getpass("Introduce la contraseña de la base de datos: ")  # Oculta la contraseña
            database = input("Introduce el nombre de la base de datos: ")

            connection_string = f"mysql+pymysql://{user}:{password}@{host}/{database}"
            self.engine = db.create_engine(connection_string)
            print("Conexión establecida correctamente.")
            self.analyze_tables()
        except SQLAlchemyError as e:
            print(f"Error al conectar a la base de datos: {e}")

    def count_nulls(self, df):
        return df.isnull().sum()

    def analyze_tables(self):
        try:
            # Proceso de limpieza para la tabla vista_datos_completos
            query_vista = "SELECT * FROM vista_datos_completos"
            df_vista = pd.read_sql(query_vista, self.engine)
            print("Valores nulos antes de la limpieza (vista_datos_completos):")
            print(self.count_nulls(df_vista))

            columns_to_check = df_vista.columns.difference(['spread', 'short_float'])
            df_vista_cleaned = df_vista.dropna(subset=columns_to_check, how='any')

            print("Valores nulos después de la limpieza (vista_datos_completos):")
            print(self.count_nulls(df_vista_cleaned))

            # Aplicar el filtro adicional a vista_datos_completos
            df_vista_cleaned = self.filter_vista_datos_completos(df_vista_cleaned)

            # Proceso de limpieza para la tabla OHLCData
            query_ohlc = "SELECT * FROM OHLCData"
            df_ohlc = pd.read_sql(query_ohlc, self.engine)
            print("Valores nulos antes de la limpieza (OHLCData):")
            print(self.count_nulls(df_ohlc))

            # Limpiar nulos en OHLCData
            df_ohlc_cleaned = df_ohlc.dropna()

            print("Valores nulos después de la limpieza (OHLCData):")
            print(self.count_nulls(df_ohlc_cleaned))

            # Obtener los id_event que cumplen con las condiciones en ambas tablas
            cleaned_events_vista = df_vista_cleaned['id_event'].unique()
            cleaned_events_ohlc = df_ohlc_cleaned['id_event'].unique()

            # Encontrar la intersección de id_event entre ambas tablas
            common_events = set(cleaned_events_vista).intersection(set(cleaned_events_ohlc))

            # Filtrar ambas tablas para mantener solo los registros con id_event comunes
            df_vista_final = df_vista_cleaned[df_vista_cleaned['id_event'].isin(common_events)]
            df_ohlc_final = df_ohlc_cleaned[df_ohlc_cleaned['id_event'].isin(common_events)]

            # Calcular el PnL_percent para cada evento en OHLCData
            df_ohlc_final = self.calculate_pnl(df_ohlc_final)

            # Fusionar la columna pnl en vista_datos_completos
            df_merged = df_vista_final.merge(
                df_ohlc_final[['id_event', 'pnl_percent']].drop_duplicates('id_event'),
                on='id_event',
                how='left'
            )

            # Guardar la tabla final
            self.cleaned_dataframes['vista_datos_completos'] = df_merged

            print("Tabla vista_datos_completos filtrada, limpia y con % pnl guardada correctamente.")
        except Exception as e:
            print(f"Error al analizar las tablas: {e}")

    def filter_vista_datos_completos(self, df):
        """
        Filtra la tabla vista_datos_completos según las condiciones:
        - precio < 20
        - float_shares < 40000000
        - market_cap < 300000000
        """
        try:
            required_columns = ['precio', 'float_shares', 'market_cap']
            if all(column in df.columns for column in required_columns):
                filtered_df = df[(df['precio'] < 15) &
                                 (df['float_shares'] < 40000000) &
                                 (df['market_cap'] < 500000000)]
                print("vista_datos_completos filtrada según las condiciones especificadas.")
                return filtered_df
            else:
                print("Advertencia: No se encontraron todas las columnas requeridas para el filtrado.")
                return df
        except Exception as e:
            print(f"Error al filtrar vista_datos_completos: {e}")
            return df

    def calculate_pnl(self, df_ohlc):
        """
        Calcula el % PnL para cada evento en el DataFrame OHLCData.
        """
        try:
            # Definir rangos para stop loss y take profit (en porcentaje)
            stop_loss_percentage = 0.05  # 3%
            take_profit_percentage = 0.04  # 4%

            # Lista para almacenar los resultados del PnL
            pnl_results = []

            # Iterar sobre cada evento único
            for event_id in df_ohlc['id_event'].unique():
                # Filtrar los datos OHLC para el evento actual
                ohlc_data = df_ohlc[df_ohlc['id_event'] == event_id]

                if ohlc_data.empty:
                    continue  # Saltar eventos sin datos

                # Obtener el precio de apertura del evento
                open_price = ohlc_data.iloc[0]['open']

                # Inicializar stop loss y take profit
                stop_loss_price = open_price * (1 - stop_loss_percentage)
                stop_loss_adjusted = open_price * 0.10  # Ajuste si sube 20%
                take_profit_price = ohlc_data['high'].max()  # Tomar el máximo como take profit

                # Determinar el resultado del trade en porcentaje
                exit_price = take_profit_price  # Default

                for _, row in ohlc_data.iterrows():
                    if row['high'] >= open_price * 1.20:
                        stop_loss_price = stop_loss_adjusted  # Ajustar stop loss dinámicamente

                    if row['low'] <= stop_loss_price:
                        exit_price = stop_loss_price
                        break
                    if row['high'] >= take_profit_price:
                        exit_price = take_profit_price
                        break

                # Calcular el PnL final
                pnl_percent = ((exit_price - open_price) / open_price) * 100

                # Guardar resultados para cada evento
                pnl_results.append({
                    'id_event': event_id,
                    'pnl_percent': pnl_percent
                })

            # Convertir los resultados a un DataFrame
            pnl_df = pd.DataFrame(pnl_results)

            return pnl_df

        except Exception as e:
            print(f"Error al calcular el % PnL: {e}")
            return df_ohlc

    def save_cleaned_data_to_hdf5(self, file_path):
        try:
            if self.cleaned_dataframes:
                # Verificar si el archivo ya existe
                if os.path.exists(file_path):
                    overwrite = input(f"El archivo {file_path} ya existe. ¿Desea sobrescribirlo? (s/n): ").strip().lower()
                    if overwrite != 's':
                        new_file_path = input("Introduce la nueva ruta del archivo HDF5: ")
                        file_path = new_file_path

                # Forzar el cierre del archivo si está abierto
                try:
                    import tables
                    if tables.is_hdf5_file(file_path):
                        with pd.HDFStore(file_path, mode='r') as store:
                            store.close()
                except Exception:
                    pass

                # Guardar los DataFrames en el archivo HDF5
                with pd.HDFStore(file_path, mode='w') as store:
                    for table_name, df in self.cleaned_dataframes.items():
                        store.put(table_name, df)
                        print(f"Tabla '{table_name}' guardada en {file_path}")
            else:
                print("No hay datos limpios para guardar.")
        except Exception as e:
            print(f"Error al guardar los DataFrames en HDF5: {e}")


# Montar Google Drive
drive.mount('/content/drive')

# Crear una instancia de DatabaseAnalyzer
analyzer = DatabaseAnalyzer()

# Establecer la conexión a la base de datos
analyzer.get_db_connection()

# Guardar los DataFrames limpios y filtrados en un archivo HDF5
file_path = '/content/drive/MyDrive/Proyecto_SmallCaps/BBDD/bbdd_filtrada.h5'
analyzer.save_cleaned_data_to_hdf5(file_path)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.0/45.0 kB 846.3 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 20.9 MB/s eta 0:00:00
Mounted at /content/drive
Introduce el host de la base de datos: librobot.org
Introduce el usuario de la base de datos: faq2108
Introduce la contraseña de la base de datos: ··········
Introduce el nombre de la base de datos: smallcaps
Conexión establecida correctamente.
Valores nulos antes de la limpieza (vista_datos_completos):
ticker            0
id_event          0
fecha             0
percent_var       0
ratio_vol         0
precio            3
volumen           0
spread          133
insercion         0
anterior          0
apertura          0
float_shares    431
exchange        238
country         238
avg_volume      238
shs_outstand    252
market_cap      240
inst_own        238
short_float     441
dtype: int64
Valores nulos después de la limpieza (vista_datos_completos):
ticker            0
id_event          0
fecha            

In [ ]:
# Para verificacion:
# Ruta al archivo HDF5
df = pd.read_hdf('/content/drive/MyDrive/Proyecto_SmallCaps/BBDD/bbdd_filtrada.h5', key='vista_datos_completos')

print(df.head())

  ticker  id_event               fecha  percent_var  ratio_vol  precio  \
0   GNLN         1 2025-01-15 14:33:00        10.26       3.52    1.72   
1   VCIG         3 2025-01-15 14:38:49        18.97       3.45    2.32   
2   LAES         4 2025-01-15 14:39:28        23.22       3.44    4.51   
3   XRTX         7 2025-01-15 14:55:55       -35.43       3.25    0.82   
4   QLGN         8 2025-01-15 14:56:32        -2.47       3.24    3.95   

    volumen  spread           insercion  anterior  apertura  float_shares  \
0        27    0.08 2025-01-15 14:32:53      1.56      1.56     1980000.0   
1       312    0.03 2025-01-15 14:38:42      1.95      2.02     7600000.0   
2  13800000    0.01 2025-01-15 14:39:21      3.66      3.97    22110000.0   
3       150    0.08 2025-01-15 14:55:48      1.27      1.22     3370000.0   
4         4    0.55 2025-01-15 14:56:25      4.05      4.02      690000.0   

  exchange      country  avg_volume  shs_outstand   market_cap  inst_own  \
0     NASD      

Merge PnL con vista de datos completos...